In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json


In [2]:
import io
# import pandas as pd
# from google.colab import files
from collections import OrderedDict
import inspect as inspector
import sys
import argparse
# import json
import time
import random
# import flask
# from flask import Flask
# from flask import request
import hashlib

In [3]:
!pip install -q "tensorflow-text"

import requests
import tensorflow as tf
import tensorflow_text as tf_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 69.8 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

!pip install transformers #==2.9.0
!pip install torch  #==2.0.0

# !pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.1.0-py3-none-any.whl

!pip install accelerate
# !pip install transformers==2.9.0 
!pip install pytorch_lightning  #==0.7.5
# !pip install datsets transformers[sentencepiece]
!pip install sentencepiece

!pip install Sentencepiece
# !pip install transformers

!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=66b6bb78a47a6f1ec296216ded1207704082080e86ac1b5c4b92e02f365875b7
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 86.0

In [5]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_validation = pd.read_json('./validation.jsonl', lines=True)

In [6]:
def concatenate_strings(lst):
    separator = ' '
    return separator.join(lst)

In [7]:
df['targetParagraphs'] = df['targetParagraphs'].apply(concatenate_strings)
df['postText'] = df['postText'].apply(concatenate_strings)
df['spoiler'] = df['spoiler'].apply(concatenate_strings)
df['tags'] = df['tags'].apply(concatenate_strings)

In [8]:
df_validation['targetParagraphs'] = df_validation['targetParagraphs'].apply(concatenate_strings)
df_validation['postText'] = df_validation['postText'].apply(concatenate_strings)
df_validation['spoiler'] = df_validation['spoiler'].apply(concatenate_strings)
df_validation['tags'] = df_validation['tags'].apply(concatenate_strings)

In [9]:
def generate_new_df(df):

    new_df = pd.DataFrame({'tags': df['tags'],'postText':df['postText'],'paragraph' : df['targetParagraphs'] ,'spoiler': df['spoiler']})
    return new_df
    

In [10]:

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



# sklearn

from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



# Utility

import re
import string
import pickle

In [11]:
wordLemm = WordNetLemmatizer()
def processed_paragraph(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [12]:
wordLemm = WordNetLemmatizer()
def processed_postText(paragraph):
    # Lower Casing
    paragraph = paragraph.lower()
    # Remove punctuations
    paragraph = paragraph.translate(str.maketrans("","",string.punctuation))
    # Tokenizing words
    tokens = word_tokenize(paragraph)
    # Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    # Reducing a word to its word stem using lemmatization
    
    return ' '.join(tokens)

In [13]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
new_df = generate_new_df(df)

In [16]:
new_df_validation = generate_new_df(df_validation)

In [17]:
 new_df['paragraph'] = new_df['paragraph'].apply(lambda x: processed_postText(x))
 new_df['postText'] = new_df['postText'].apply(lambda x: processed_postText(x))
# print('Text Preprocessing complete.')

In [18]:
 new_df_validation['paragraph'] = new_df_validation['paragraph'].apply(lambda x: processed_postText(x))
 new_df_validation['postText'] = new_df_validation['postText'].apply(lambda x: processed_postText(x))

In [19]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
questions= new_df['postText']
answers= new_df['spoiler']
stories_not_summarized = new_df['paragraph']
tags = new_df['tags']

print(stories_not_summarized[0])




it ’ ll be just like old times this weekend for tom brady and wes welker welker revealed friday morning on a miami radio station that he contacted brady because he ’ ll be in town for sunday ’ s game between the new england patriots and miami dolphins at gillette stadium it seemed like a perfect opportunity for the two to catch up but brady ’ s definition of catching up involves far more than just a meal in fact it involves some literal catching as the patriots quarterback looks to stay sharp during his fourgame deflategate suspension i hit him up to do dinner saturday night he ’ s like ‘ i ’ m going to be flying in from ann arbor later after the michigancolorado football game but how about that morning we go throw ’ welker said on wqam per the boston globe and i ’ m just sitting there i ’ m like ‘ i was just thinking about dinner but yeah sure i ’ ll get over there early and we can throw a little bit ’ welker was one of brady ’ s favorite targets for six seasons from 2007 to 2012 it ’

In [21]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=100918e2f7ec4cddf689707bde0b457f8728368461e5270e7a7d962c18905804
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [36]:
from transformers import pipeline
import requests
import pprint
import time
pp = pprint.PrettyPrinter(indent=14)

from transformers import T5Tokenizer, T5ForConditionalGeneration
import wandb

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [37]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [38]:
from datetime import datetime

In [39]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="bart_wiki_lingua",
        config={
            "per_device_train_batch_size": 4
            # "learning_rate": training_args.learning_rate
            # "dataset": "wiki_lingua " + "english",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + "english" + "_" + current_time

In [40]:
# summarize with T5
# summarizer_t5 = pipeline(task='summarization', model="t5-large")
summarizer_t5 = pipeline(task='summarization', model="t5-small")

In [ ]:
# # total = range(spoiler.size)
# total = range(9)
# for i in total:
#   input_text1 = listToString(targetParagraphs[i])
#   input_text1

#   input_text = input_text1
#   # for par in targetParagraphs[1]
#   #   input_text += par

#   t0 = time.time() # timer
#   if tags[i][0] == "phrase":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=4, max_length=15) # change min_ and max_length for different output
#   elif tags[i][0] == "passage":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=15, max_length=50) # change min_ and max_length for different output

#   elif tags[i][0] == "multi":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=50, max_length=500) # change min_ and max_length for different output

#   print(summary_manifesto_t5)
#   print(spoiler[i])
#   print("Summarization took " + str(round((time.time() - t0) / 60, 2)) + " minutes.") # timer

In [ ]:
# from tqdm.autonotebook import tqdm
# total1 = len(stories_not_summarized)
# summary_manifestos_t5 = []
# for i in tqdm(range(total1)): 
#     stories= stories_not_summarized[i][:1000]
#     summary_manifesto_t5 = summarizer_t5(stories, min_length=4, max_length=500)
#     summary_manifestos_t5.append(summary_manifesto_t5)

In [ ]:
# # total = range(spoiler.size)
# from tqdm.autonotebook import tqdm
# total1 = len(stories_not_summarized)
# summary_manifestos_t5 = []
# total = range(9)
# for i in total:
#   input_text1 = listToString(targetParagraphs[i])
#   input_text1

#   input_text = input_text1
#   # for par in targetParagraphs[1]
#   #   input_text += par

#   t0 = time.time() # timer
#   if tags[i][0] == "phrase":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=4, max_length=15) # change min_ and max_length for different output
#   elif tags[i][0] == "passage":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=15, max_length=50) # change min_ and max_length for different output

#   elif tags[i][0] == "multi":
#     summary_manifesto_t5 = summarizer_t5(input_text, min_length=50, max_length=500) # change min_ and max_length for different output

#   print(summary_manifesto_t5)
#   print(spoiler[i])
#   print("Summarization took " + str(round((time.time() - t0) / 60, 2)) + " minutes.") # timer

In [42]:
from tqdm.autonotebook import tqdm
total1 = len(stories_not_summarized)
# total1 = len(40)
summary_manifestos_t5 = []
for i in tqdm(range(total1)): 
    stories= stories_not_summarized[i][:1000]
    # summary_manifesto_t5 = summarizer_t5(stories, min_length=4, max_length=500)
    # summary_manifesto_bart = summarizer(input_text, min_length=4, max_length=15) # change min_ and max_length for different output


    if tags[i] == "phrase":
      summary_manifesto_bart = summarizer_t5("Summarize:" + stories, min_length=4, max_length=15) # change min_ and max_length for different output
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)
    elif tags[i] == "passage":
      summary_manifesto_bart = summarizer_t5("Summarize:" + stories, min_length=15, max_length=50) # change min_ and max_length for different output
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)
    elif tags[i] == "multi":
      summary_manifesto_bart = summarizer_t5("Summarize:" + stories, min_length=50, max_length=500) # change min_ and max_length for different output
      summary_manifestos_t5.append(summary_manifesto_bart)
      print(summary_manifesto_bart)

    # summary_manifesto_bart = summarizer_bart(stories, min_length=4, max_length=500) # change min_ and max_length for different output
    # summary_manifestos_t5.append(summary_manifesto_bart)
    # print("\n")
    # print(summary_manifesto_bart)
    # print("\n")
    # print(tags[i])

    # summary_manifestos_t5.append(summary_manifesto_bart)

file = open("T5_results.txt", "w")

#convert variable to string
str_1 = repr(summary_manifestos_t5)
file.write("T5_results = " + str_1 + "\n")

#close file
file.close()

  0%|          | 0/3200 [00:00<?, ?it/s]

[{'summary_text': "wes welker contacted tom brady because he ' ll be in town for sunday's game between the new england patriots and miami dolphins at gillette stadium"}]
[{'summary_text': 'nasa scientists are predicting the hole in the '}]


Your max_length is set to 500, but your input_length is only 226. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=113)


[{'summary_text': 'research by hiring software provider cangrade revealed that being intellectually stimulated'}]
[{'summary_text': "cal newport' s investigation into how passionate people like steve jobs really got started as well as what scientists say predicts happiness and fuels great accomplishment newport . ryan holiday author of ego is the enemy your passion may be the very enemy . the only way to do great work is to love what you do if you haven't found it yet keep looking dont settle in other words follow your passion there ’ s just one problem . ‘follow your passion ’ is dangerous advice that ’"}]
[{'summary_text': 'the best way to cook rice is in a rice cooker which you'}]
[{'summary_text': 'apple iphone event introduced airpods apple totally wireless headphones . one of the biggest surprise announcements was the introduction of airpod . apple says youll be able to buy just one apple didnt say how much it would cost to'}]
[{'summary_text': 'gabor kiraly has made a name for h

Your max_length is set to 500, but your input_length is only 239. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=119)


...

[{'summary_text': 'terry kniess worked as a weatherman in las vegas and learned to recognize patterns to predict the weather . he also worked at a casino helping catch card counters his wife is a mathematician after friends told them how fun it was being on the show they decided to make it their goal .'}]


Your max_length is set to 500, but your input_length is only 272. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=136)


[{'summary_text': 'i love rainymood for some soothing white noise you can download it as an app but i like to just have it open in a tab 2 if you have 5 minutes put pressure on period cramps whatever helps right 3 . i always carry around an essentialoil rollerball in my purse to rub on my pulse points im a huge fan of candles and scented baths .'}]
[{'summary_text': 'some weird things to try out in the kitchen recommended by reddit users . mcdonalds hashbrowns beavercleever peanut butter on beef burger angus beef is delicious attheeolian strawberry jam on a plain bagel .'}]
[{'summary_text': 'researchers kept track of 77000 men and women over the course of seven years . they found that after seven years there were 380 colon cancer cases and 110 rectal cancer cases among the group .'}]
[{'summary_text': 'faye lee millie found out she was pregnant and she and her partner lewis were completely thrilled but when they got the results of her ultrasound they learned she would be having twins 

Your max_length is set to 500, but your input_length is only 219. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=109)


[{'summary_text': "rance quinn and a coworker went out to a chili ' s for lunch and were surprised by a teenager approaching them she laid a napkin on the table and walked away before anyone could say"}]
[{'summary_text': 'antiaging products and routines are marketed at people mostly women in their 20s 30s 40s and 50s use antiwrinkle cream sleep with tea bags under your eyes avoid stress they command . the myth antiaging creams and lotions will roll back the clock and make fine lines disappear why its bogus according to staff at the mayo clinic .'}]
[{'summary_text': 'beyoncefan666 tweeted that beyoncé was'}]
[{'summary_text': 'charlie dent rpa came out in support of'}]


In [46]:
stories = summary_manifestos_t5

In [47]:
stories_list =[]
for i in range(len(stories)):
    summary=stories[i][0]
    stories_list.append(summary['summary_text'])


print(stories_list)

["wes welker contacted tom brady because he ' ll be in town for sunday's game between the new england patriots and miami dolphins at gillette stadium", 'nasa scientists are predicting the hole in the ', 'research by hiring software provider cangrade revealed that being intellectually stimulated', "cal newport' s investigation into how passionate people like steve jobs really got started as well as what scientists say predicts happiness and fuels great accomplishment newport . ryan holiday author of ego is the enemy your passion may be the very enemy . the only way to do great work is to love what you do if you haven't found it yet keep looking dont settle in other words follow your passion there ’ s just one problem . ‘follow your passion ’ is dangerous advice that ’", 'the best way to cook rice is in a rice cooker which you', 'apple iphone event introduced airpods apple totally wireless headphones . one of the biggest surprise announcements was the introduction of airpod . apple says 

In [ ]:
# from transformers import BartTokenizer, BartModel #, BartForCausalLM #,  BartModel

# tokenizer_bart = BartTokenizer.from_pretrained('facebook/bart-large')
# model_bart = BartModel.from_pretrained('BartForQuestionAnswering')

In [39]:
# from transformers import pipeline
# qa_pipleline = pipeline('question-answering', model=BartForQuestionAnswering, tokenizer=tokenizer, device=0)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'BartForQuestionAnswering' is not defined

In [ ]:
# answers =[]
# for i in tqdm(range(len(questions))):
#   answer = qa_pipleline({
#        'question': questions[i],
#         'context': stories_list[i]
#   })
#   answers.append(answer)

In [ ]:
# ans=[]
# for i in range(len(answers)):
#   ans.append(answers[i]['answer'])
#   print(answers[i]['answer'])

In [48]:
type(new_df['spoiler'][0:40])

pandas.core.series.Series

In [49]:
type(stories_list[0:40])

list

In [50]:
stories_list_series = pd.Series((v for v in stories_list))

In [51]:
import pandas as pd

In [53]:
dataframe = pd.DataFrame({ 'postText': questions, 'actual_spoiler':new_df['spoiler'], 'gen_spoiler':stories_list_series })

In [52]:
if WANDB_INTEGRATION:
    wandb_run.finish()

In [ ]:
#EVALUATION METRICS

In [54]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

In [55]:
bleu_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  score_ref = bleu.sentence_bleu([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"])
  bleu_scores.append(score_ref)



In [56]:
bleu_scores

[2.247804721170705e-79,
 0,
 0.05038781936709891,
 0.20553293037748185,
 0.08737902619542039,
 0.17237363755062024,
 0.03220700123472736,
 0.09103131378420805,
 0.28096425394598984,
 0.018400391398450698,
 0.05815280688479283,
 2.8468306583110405e-82,
 1.3369241227685064e-155,
 0.24791216857277443,
 0.17976304758609282,
 0.5484246634790253,
 2.198520996694053e-78,
 0.003759171746412077,
 0.13868656466721938,
 0.2417520931822434,
 0.0004139109093188785,
 0.14517736316508584,
 0.4160161901819732,
 0.01650606420712778,
 0.016420116735599958,
 2.0271391852593266e-78,
 0.42726392790084855,
 0.009925085655342007,
 0.08606449149533135,
 0.0002138440798129263,
 0.0026285583503802795,
 0.32389946531722097,
 0.0209307337714295,
 0.014730511857675259,
 0.045341908684083804,
 1.2868286781332451e-155,
 5.316705693944092e-233,
 0.04591680138433917,
 5.606611650548264e-156,
 0.092549706109518,
 0.049787068367863944,
 0.0006551938938599568,
 0.011108996538242306,
 0.24940521645000488,
 0.0046205207929

In [57]:
bleu_average = round(sum(bleu_scores) / len(bleu_scores), 2)
bleu_average

0.11

In [58]:
bleu_min = min(bleu_scores)
bleu_min

0

In [59]:
bleu_max = max(bleu_scores)
bleu_max

0.9189987133950746

In [60]:
bleu_scores.sort()
mid = len(bleu_scores) // 2
res = (bleu_scores[mid] + bleu_scores[~mid]) / 2
bleu_median = res
bleu_median

0.03980267159475036

In [61]:
import sys
from nltk.translate.meteor_score import meteor_score

In [62]:
meteor_scores = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"].strip()

  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(dataframe.iloc[i]["actual_spoiler"].strip())
  meteor = nltk.translate.meteor_score.meteor_score([ref], hypo)


  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  meteor_scores.append(meteor)



In [63]:
meteor_scores

[0.019920318725099598,
 0.0,
 0.18564356435643564,
 0.13023782559456398,
 0.3279958677685951,
 0.16506493506493508,
 0.041666666666666664,
 0.32502775379301835,
 0.20232371794871792,
 0.01288659793814433,
 0.0,
 0.0,
 0.0,
 0.40527694247744683,
 0.04143646408839779,
 0.5921052631578947,
 0.0,
 0.03105590062111801,
 0.08670520231213873,
 0.16463730569948187,
 0.058775605214152704,
 0.048859934853420196,
 0.5682931608857534,
 0.06369426751592357,
 0.03215434083601286,
 0.0,
 0.31652186900998236,
 0.11871085067690443,
 0.06006006006006006,
 0.027247956403269755,
 0.0684931506849315,
 0.1639344262295082,
 0.049019607843137254,
 0.04166666666666667,
 0.0375,
 0.0,
 0.0,
 0.05081300813008131,
 0.0,
 0.10689655172413795,
 0.2992514970059881,
 0.015576323987538939,
 0.25233920282914607,
 0.1178358321215464,
 0.043859649122807015,
 0.0,
 0.0,
 0.18564356435643564,
 0.0909090909090909,
 0.34742838802856457,
 0.05305039787798409,
 0.09803921568627452,
 0.4934210526315789,
 0.06191950464396285,
 0

In [64]:
meteor_scores_average = round(sum(meteor_scores) / len(meteor_scores), 2)
meteor_scores_average

0.13

In [65]:
meteor_scores_max = max(meteor_scores)
meteor_scores_max

0.9921875

In [66]:
meteor_scores_min = min(meteor_scores)
meteor_scores_min

0.0

In [67]:
meteor_scores.sort()
mid = len(meteor_scores) // 2
res = (meteor_scores[mid] + meteor_scores[~mid]) / 2
meteor_scores_median = res
meteor_scores_median

0.06410256410256411

In [68]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00


In [61]:
from bert_score import score

In [62]:
!git clone https://github.com/Tiiiger/bert_score.git

Cloning into 'bert_score'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 993 (delta 219), reused 322 (delta 198), pack-reused 638
Receiving objects: 100% (993/993), 1.34 MiB | 10.33 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [75]:
bert_scores_P = []
bert_scores_R = []
bert_scores_F1 = []
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  P, R, F1 = score(gen_spoiler_strip, actual_spoiler_strip, lang="en", verbose=True)
  bert_scores_P.append(P)
  bert_scores_R.append(R)
  bert_scores_F1.append(F1)

  # meteor = nltk.translate.meteor_score.meteor_score([gen_spoiler_strip], dataframe.iloc[i]["actual_spoiler"].strip())
  # meteor = round(meteor_score([dataframe.iloc[i]["gen_spoiler"]], dataframe.iloc[i]["actual_spoiler"]), 2) # list of references
  # meteor_scores.append(meteor)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:13                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/bert_score/score.py:70 in score                          │
│                                                                                                  │
│    67 │   │   │   │     multiple references, the returned score of this candidate is             │
│    68 │   │   │   │     the *best* score among all references.                                   │
│    69 │   """                                                                                    │
│ ❱  70 │   assert len(cands) == len(refs), "Different number of candidates and references"        │
│    71 │                                                                                          │
│    72 │   assert (                                                                               │
│    73 │   │   lang is not None or model_type is not None                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: Different number of candidates and references

In [ ]:
# P, R, F1 = score(cands, refs, lang="en", verbose=True)

In [74]:
for i in range(len(dataframe)):
# for data_results in dataframe[0]:
  gen_spoiler_strip = dataframe.iloc[i]["gen_spoiler"]
  actual_spoiler_strip = dataframe.iloc[i]["actual_spoiler"]


  ref = word_tokenize(gen_spoiler_strip)
  hypo = word_tokenize(actual_spoiler_strip)

  print("\n")
  print(ref)
  print("\n")
  print(hypo)
  print("\n")



['tom', 'brady', "'", 's', 'definition', 'of', 'catching', 'up', 'involves', 'much', 'more', 'than', 'just', 'a', 'meal', '.', 'wes', 'welker', 'contacted', 'the', 'patriots', 'quarterback', 'friday', 'morning', 'on', 'a', 'miami', 'radio', 'station', '.', 'bra']


['how', 'about', 'that', 'morning', 'we', 'go', 'throw', '?']




['nasa', 'scientists', 'are', 'predicting', 'the', 'hole', 'in', 'the']


['2070']




['research', 'by', 'hiring', 'software', 'provider', 'cangrade', 'found', 'that', 'being', 'intellectually', 'stimulated']


['intellectual', 'stimulation']




['cal', 'newport', "'", 's', 'investigation', 'into', 'how', 'passionate', 'people', 'like', 'steve', 'jobs', 'really', 'got', 'started', 'as', 'well', 'as', 'what', 'scientists', 'say', 'predicts', 'happiness', 'and', 'fuels', 'great', 'accomplishment', 'newport', '.', 'ryan', 'holiday', 'author', 'of', 'ego', 'is', 'the', 'enemy', 'your', 'passion', 'may', 'be', 'the', 'very', 'enemy', '.', 'the', 'only', 'way', 

In [ ]:
bert_scores_P

In [ ]:
bert_scores_R

In [ ]:
bert_scores_F1